# **CH02.4. Transformer**

#### **`Paper Info`** : Attention Is All You Need
#### $ \hspace{1.75cm} - $ Ashish Vaswani, Noam Shazeer, Niki Parmar et al. (NeurIPS/2017)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|------|
| Research topic | RNN/CNN 없이 Self-Attention으로만 구성된 Transformer를 제안하여 sequence transduction 품질과 학습 효율을 동시 달성 |
| Core idea | Encoder–Decoder를 Multi-Head Self-Attention과 Position-wise FFN으로 구성하고 Positional Encoding으로 순서 정보를 부여하며 <br> Masked Attention으로 autoregressive decoding을 구현 |
| Key findings | $ \cdot{} $ WMT14 En $ \rightarrow{} $ De, En $ \rightarrow{} $ Fr에서 SOTA 수준 BLEU 달성 <br> $ \cdot{} $ GPU에서 Attention의 행렬곱 병렬화로 학습 속도 향상 <br> $ \cdot{} $ Label smoothing, weight sharing, warmup scheduler로 안정적 학습 확보 |
| Contributions | $ \cdot{} $ Recurrence를 제거한 최초의 순수 Attention 기반 아키텍처 제안 <br> $ \cdot{} $ Scaled Dot-Product와 Multi-Head 설계 정식화 <br> $ \cdot{} $ Sinusoidal Positional Encoding을 통한 길이 외삽 가능성 제시 <br> $ \cdot{} $ 입력/출력 임베딩과 pre-softmax 투영을 공유하는 weight tying로 파라미터 효율 개선 |

<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ RNN/LSTM 계열은 시점 의존적 계산으로 병렬화에 제약이 커 긴 시퀀스에서 학습과 추론 효율이 낮음
##### $ \hspace{0.15cm} \cdot{} $ CNN은 병렬화는 가능하나 멀리 떨어진 토큰 의존을 포착하려면 큰 receptive field가 필요하며 깊이와 커널 확장이 요구됨
##### $ \hspace{0.15cm} \cdot{} $ RNN 기반 seq2seq는 고정 길이 context bottleneck을 Attention으로 완화했지만 여전히 순차 계산의 제약을 벗어나지 못함
#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 장거리 의존을 경로 길이 증가 없이 직접 학습하여 번역 품질을 높이는 sequence transduction 달성
##### $ \hspace{0.15cm} \cdot{} $ 시점 병렬화를 극대화하여 학습 속도와 하드웨어 효율을 개선
##### $ \hspace{0.15cm} \cdot{} $ 대규모 데이터셋에서 재현 가능한 안정적 학습 절차 확립
#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 순수 Self-Attention 기반 Encoder–Decoder로 recurrence와 convolution을 제거하여 병렬성 극대화를 의도
##### $ \hspace{0.15cm} \cdot{} $ Scaled Dot-Product와 Multi-Head 설계로 다양한 서브스페이스 의존을 병렬 포착하도록 의도
##### $ \hspace{0.15cm} \cdot{} $ Sinusoidal Positional Encoding으로 순서 정보를 주입하고 길이 외삽성을 확보하도록 의도
##### $ \hspace{0.15cm} \cdot{} $ Masked Self-Attention으로 autoregressive decoding의 정합성과 안정성을 확보하도록 의도

<b></b>

> ## **방법론(Method)**

#### **(1) 아키텍처 개요** :
##### $ \hspace{0.15cm} \cdot{} $ Encoder $ N $층 스택과 Decoder $ N $층 스택으로 구성되며 각 sub-layer 뒤에 residual connection과 LayerNorm 적용
##### $ \hspace{0.15cm} \cdot{} $ Encoder-layer: Multi-Head Self-Attention $\rightarrow{}$ Position-wise FFN
##### $ \hspace{0.15cm} \cdot{} $ Decoder-layer: Masked Multi-Head Self-Attention $\rightarrow{}$ Encoder–Decoder Attention $\rightarrow{}$ Position-wise FFN
##### **(`PLUS`) 파라미터 스케일** :
##### $ \hspace{0.15cm} \cdot{} $ Base의 전형적 설정은 $ d_{\text{model}} = 512, \, d_{\text{ff}} = 2048, \, h = 8, \, N = 6 $
#### **(2) Scaled Dot-Product Attention** :
##### $ \hspace{0.15cm} \cdot{} $ 입력 $ Q \in{} \mathbb{R}^{t_{q} \times d_{k}}, \, K \in{} \mathbb{R}^{t_{k} \times d_{k}}, \, V \in{} \mathbb{R}^{t_{k} \times d_{v}} $
##### $ \hspace{0.15cm} \Rightarrow{} \text{Attention}(Q,K,V) = \text{softmax}\big({} (QK^{\top}) / \sqrt{d_{k}} \big) V \;\; \text{ where } \, d_{k} : \text{key/query dimension}, \;\; d_{v} : \text{value dimension} $
##### $ \hspace{0.15cm} \cdot{} $ $ \sqrt{d_{k}} $로의 스케일링은 $ q \cdot{} k $의 분산이 $ d_{k} $에 비례한다는 가정 하 softmax 포화를 완화하기 위함
##### **(`PLUS`) 마스킹의 행렬 구현** :
##### $ \hspace{0.15cm} \Rightarrow{} \tilde{S} = (QK^{\top}) / \sqrt{d_{k}} + M \;\; \text{ where } \, M_{ij} = -\infty{} \, \text{ if future is illegal, else } 0 $
#### **(3) Multi-Head Attention** :
##### $ \hspace{0.15cm} \cdot{} $ 각 head $ i $에 대해 $ W_{i}^{Q} \in{} \mathbb{R}^{d_{\text{model}} \times d_{k}}, \, W_{i}^{K} \in{} \mathbb{R}^{d_{\text{model}} \times d_{k}}, \, W_{i}^{V} \in{} \mathbb{R}^{d_{\text{model}} \times d_{v}} $
##### $ \hspace{0.15cm} \Rightarrow{} \text{head}_{i} = \text{Attention}(X W_{i}^{Q}, X W_{i}^{K}, X W_{i}^{V}) \;\; \text{ where } \, X \in{} \mathbb{R}^{t \times d_{\text{model}}} $
##### $ \hspace{0.15cm} \Rightarrow{} \text{MultiHead}(X) = \text{Concat}(\text{head}_{1}, \ldots{}, \text{head}_{h}) W^{O} \;\; \text{ where } \, W^{O} \in{} \mathbb{R}^{(h d_{v}) \times d_{\text{model}}} $
##### $ \hspace{0.15cm} \cdot{} $ 서로 다른 표현 서브스페이스에서 상이한 의존을 병렬로 포착하여 단일 head 평균화로 인한 해상도 손실을 방지
#### **(4) Positional Encoding** :
##### $ \hspace{0.15cm} \cdot{} $ 사인/코사인 기반 고정식 인코딩으로 각 차원 주파수를 기하급수적 스케일로 배치
##### $ \hspace{0.15cm} \Rightarrow{} \text{PE}(pos, 2i) = \sin{}\big({} pos / 10000^{2i/d_{\text{model}}} \big), \;\; \text{PE}(pos, 2i{+}1) = \cos{}\big({} pos / 10000^{2i/d_{\text{model}}} \big) $
##### $ \hspace{0.15cm} \cdot{} $ $ \sin{}(\omega(pos{+}k)) $ 항등식으로부터 상대 위치 이동이 선형 변환으로 표현 가능하여 길이 외삽에 유리
##### **(`PLUS`) Learned positional embedding과 성능은 유사하나 외삽성 관점에서 sinusoid 선택**
#### **(5) Decoder의 Masked Self-Attention** :
##### $ \hspace{0.15cm} \cdot{} $ upper-triangular mask로 $ j > i $ 위치의 로짓을 $ -\infty{} $ 처리하여 softmax 후 확률 $ 0 $이 되도록 함
##### $ \hspace{0.15cm} \cdot{} $ teacher forcing 훈련과 autoregressive 추론의 정합성 보장
#### **(6) Position-wise Feed-Forward Network(FFN)** :
##### $ \hspace{0.15cm} \cdot{} $ 각 위치에 독립적으로 동일한 MLP 적용
##### $ \hspace{0.15cm} \Rightarrow{} \text{FFN}(x) = \max{}(0, x W_{1} + b_{1}) W_{2} + b_{2} \;\; \text{ where } \, W_{1} \in{} \mathbb{R}^{d_{\text{model}} \times d_{\text{ff}}}, \, W_{2} \in{} \mathbb{R}^{d_{\text{ff}} \times d_{\text{model}}} $
##### **(`PLUS`) 1x1 convolution으로 해석 가능하며 상호작용은 Attention이 담당**
#### **(7) 출력층과 임베딩 공유 및 스케일링** :
##### $ \hspace{0.15cm} \cdot{} $ 입력/출력 임베딩과 pre-softmax 가중치를 공유하여 파라미터 효율성 확보
##### $ \hspace{0.15cm} \cdot{} $ 입력 임베딩에 $ \sqrt{d_{\text{model}}} $ 스케일을 곱해 크기 정규화
#### **(8) 최적화와 정규화 스킴** :
##### $ \hspace{0.15cm} \cdot{} $ Optimizer: Adam with $ \beta_{1} = 0.9, \, \beta_{2} = 0.98, \, \epsilon{} = 10^{-9} $
##### $ \hspace{0.15cm} \Rightarrow{} \text{lrate} = d_{\text{model}}^{-1/2} \cdot \min{}\big( \text{step}^{-1/2}, \, \text{step} \cdot \text{warmup}^{-3/2} \big) \;\; \text{ where } \, \text{warmup} \approx 4000 $
##### $ \hspace{0.15cm} \cdot{} $ Dropout을 sub-layer 출력과 embedding$+$positional 합에 적용
##### $ \hspace{0.15cm} \cdot{} $ Label smoothing $ \epsilon{}_{\text{ls}} \approx 0.1 $로 overconfidence를 완화


<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | WMT14 En-De (4.5M pairs), WMT14 En-Fr (36M pairs) |
| Data(Sentence;row) size | En-De: 4.5M, En-Fr: 36M |
| Vocabulary(token) | BPE/shared ~37k (En-De), word-piece ~32k (En-Fr) |
| Evaluation dataset | WMT14 test sets |
| Evaluation size | En-De: 3003 sents (?), En-Fr: 3003 sents (?) |
##### $ \hspace{0.15cm} \cdot{} $ 모델 및 훈련
| Item | Description |
|------|--------------|
| Model type | Transformer (Encoder–Decoder, Self-Attention only) |
| Hidden layer | FFN with $ d_{\text{ff}} = 2048 $ |
| Embedding dimension | $ d_{\text{model}} = 512 $ |
| Context window size | Full self-attention within sequence |
| Optimizer | Adam($ \beta_{1}=0.9, \beta_{2}=0.98, \epsilon=1e{-}9 $) |
| Learning rate | $ d_{\text{model}}^{-1/2} \cdot \min{}\big(\text{step}^{-1/2}, \, \text{step}\cdot \text{warmup}^{-3/2}\big) $, warmup $=4000 $ |
| Training hardware | 8 $\times$ NVIDIA P100 |
| Training time | Base: $ \approx 12 $h / 100k steps, Big: $ \approx 3.5 $d / 300k steps |
| Baseline models | GNMT/RNN, ConvS2S, etc |
| Evaluation metric | BLEU |
#### **(2) 분석 및 결과** :
##### $ \hspace{0.15cm} \cdot{} $ En$\rightarrow{}$De와 En$\rightarrow{}$Fr에서 단일 모델로 기존 대비 높은 BLEU를 달성
##### $ \hspace{0.15cm} \cdot{} $ Multi-Head 수와 $ d_{k} $ 축소는 성능 저하 경향이며 적정 head 수가 존재
##### $ \hspace{0.15cm} \cdot{} $ Positional Encoding의 learned vs sinusoid는 성능 유사하나 외삽성 관점에서 sinusoid 채택
##### $ \hspace{0.15cm} \cdot{} $ Parsing 등 전이 태스크에서도 경쟁력 확인
#### **(3) 결과 해석** :
##### $ \hspace{0.15cm} \cdot{} $ Self-Attention은 모든 위치 쌍을 한 층 내에서 직접 연결하여 경로 길이를 상수로 유지해 장거리 의존 학습에 유리
##### $ \hspace{0.15cm} \cdot{} $ 행렬곱 기반 병렬화로 하드웨어 효율이 높아 동일 시간 대비 더 큰 batch와 더 빠른 수렴을 달성
##### $ \hspace{0.15cm} \cdot{} $ Label smoothing과 warmup scheduler는 안정적 최적화에 기여

<b></b>

> ## **결론(Conclusion)**

#### **(1)** Transformer는 recurrence와 convolution 없이 Self-Attention만으로 sequence transduction의 성능과 효율을 동시에 달성
#### **(2)** Multi-Head, Positional Encoding, Masked Attention 등 설계 조합이 재현 가능한 학습 안정성과 SOTA 품질을 뒷받침
#### **(3)** 병렬화 이점과 장거리 의존 처리 능력으로 이후 NLP 모델 패러다임의 표준을 형성

<b></b>